# Welcome to the 
```
   _  _____  ___  ___  _____________  ____  _  __
  / |/ / _ \/ _ \/ _ \/  _/_  __/ _ \/ __ \/ |/ /
 /    / , _/ // / // // /  / / / , _/ /_/ /    / 
/_/|_/_/|_/____/____/___/ /_/ /_/|_|\____/_/|_/  
                                                 
```                                              

<i>"Let's do some science."</i>



Wait, why's this a Jupyter notebook? Great question - I'm not sure.

In [103]:
%matplotlib nbagg

import matplotlib.pyplot as plt

from mendeleev import element
from thermo.chemical import Chemical
import sympy.physics.units as units
import scipy.constants as constants
amu = constants.physical_constants["atomic mass unit-kilogram relationship"][0]
import math
import sys
import numpy as np

sys.path.insert(0, '../../files/ionprinter/simulation/utilities/')
import util


# Inputs

In [104]:
#GLOBALS

print_species = element('Aluminum')

print_speed_g_h = 10.0 #g/hour

#convert to kg/s, kick CGS to the curb.
print_speed = print_speed_g_h/1000.0
print_speed /= 3600.0

number_of_bowties = 20000.0 #bowties/printer - an excellent unit

#hold or cut bowties

build_platform_distance = 0.03 #m

#BOWTIE

bowtie_nozzle_radius = 0.00033#m

bowtie_hot_area = 1e-6 #m^2
graphite_emissivity = 0.95

bowtie_temperature = 1400.0 #Kelvin


#CHAMBER

ionization_chamber_radius = 0.003 #m
ionization_chamber_length = 0.01 #m

acceleration_gap = 0.00025 #m
acceleration_voltage = 80 #V
ionization_efficiency = 0.3 

ICP_frequency = 1000000 #hz
ICP_peak_B = 0.05 #T


# import pandas as pd
# data = [[1, 2], [3, 4]]
# pd.DataFrame(data, columns=["Foo", "Bar"])


# Neutral stuff.

In [105]:
#Total n of atoms that must be emitted to achieve desired mass deposition per hour
atoms_per_second = print_speed/(print_species.mass*amu)

moles_per_second = atoms_per_second/constants.Avogadro

#Volume deposited per hour.
hourly_print_volume = ((print_speed*3600.0)/(print_species.density*1000.0))/1.0e-6

total_beam_current = atoms_per_second*constants.elementary_charge
per_chamber_beam_current = total_beam_current/number_of_bowties

print("Atoms per second: \nper chamber %0.5g atoms/s \ntotal %0.5g atoms/s\n" % ((atoms_per_second/number_of_bowties),
                                           atoms_per_second))
print("Total moles per second: %0.5g M/s" % moles_per_second)
print("Deposited volume: %0.5g cm^3/h" % hourly_print_volume)
print("Total beam current: %0.5g Amps" % total_beam_current)
print("Per-chamber current: %0.5g Amps" % per_chamber_beam_current)

#I find it infinitely amusing that "40 amps of aluminum" is "40 grams per second of aluminum" within 0.6%

Atoms per second: 
per chamber 3.0999e+15 atoms/s 
total 6.1999e+19 atoms/s

Total moles per second: 0.00010295 M/s
Deposited volume: 3.7052 cm^3/h
Total beam current: 9.9333 Amps
Per-chamber current: 0.00049666 Amps


## RMS neutral velocity

Determine RMS neutral gas velocity at a certain temperature:

$$\sqrt{\frac{3RT}{m}}$$

In [106]:
bowtie_rms_neutral_velocity = math.sqrt((3*constants.Boltzmann*bowtie_temperature)/(print_species.mass*amu))
bowtie_rms_energy = 0.5*print_species.mass*amu*(bowtie_rms_neutral_velocity**2.0)
print("RMS particle velocity: %0.5g m/s" % bowtie_rms_neutral_velocity)
print("RMS particle energy: %0.5g eV" % (bowtie_rms_energy/constants.electron_volt))

RMS particle velocity: 1137.6 m/s
RMS particle energy: 0.18096 eV


## Bowtie nozzle gas pressure

Determine approximate neutral gas pressure in a virtual volume "1 thermal velocity" long via a simple 

$$\text{PV}=\text{nRT}$$

(I'm not totally sure that you can do this.)

Also approximately determine mean free path with the neutral Van Der Waals radius.

In [107]:
R_constant = constants.Boltzmann*constants.Avogadro

#volume nozzle area times 1 thermal neutral injection velocity long
nozzle_virtual_volume = (math.pi*(bowtie_nozzle_radius**2))*bowtie_rms_neutral_velocity

bowtie_nozzle_pressure = ((moles_per_second/number_of_bowties)*R_constant*bowtie_temperature)/(nozzle_virtual_volume)

mean_free_path = 1.0/(((atoms_per_second/number_of_bowties)/nozzle_virtual_volume)*math.pi*((print_species.vdw_radius*1e-12)**2))


print("Bowtie nozzle pressure: %0.5g Pa" % bowtie_nozzle_pressure)
print("Approximate nozzle mean free path: %0.5g m" % mean_free_path)

#number density of atoms within that virtual volume - useful for DSMC stuff
print("nrho: %0.5g" % (atoms_per_second/nozzle_virtual_volume))

print("Knudsen number: local %0.5g, global %0.5g" % ((mean_free_path/bowtie_nozzle_radius),
                                                     (mean_free_path/build_platform_distance)))

print("Worst-case space charge density: %0.5g" % ((atoms_per_second/number_of_bowties)/nozzle_virtual_volume))

Bowtie nozzle pressure: 0.15395 Pa
Approximate nozzle mean free path: 1.1805 m
nrho: 1.5929e+23
Knudsen number: local 3577.1, global 39.348
Worst-case space charge density: 7.9646e+18


## Actual vapor pressure

Now a rough approximation of the actual print species vapor pressure one can expect to find at that temperature:

(likely off by at least 50% - https://www.iap.tuwien.ac.at/www/surface/vapor_pressure is a more accurate source)

In [108]:
vapor_pressure = Chemical(print_species.name).VaporPressure.calculate(bowtie_temperature,"BOILING_CRITICAL") # takes temp in K, returns pressure in P
print("Approx. species vapor pressure: %0.5g Pa" % vapor_pressure)

Approx. species vapor pressure: 0.47493 Pa


## Bowtie radiation power loss

Bowtie IR/Vis radiation power by the Stefan-Boltzmann law:

In [109]:
per_bowtie_power = (bowtie_hot_area * graphite_emissivity * constants.Stefan_Boltzmann * (bowtie_temperature**4.0))
total_bowtie_power = per_bowtie_power*number_of_bowties

print("Bowtie power: per %0.5g W, total %0.5g W" % (per_bowtie_power,total_bowtie_power))


Bowtie power: per 0.20694 W, total 4138.8 W


# Ionization

Simple first-ionization power consumption estimate

In [110]:
per_chamber_ionization_power = ((atoms_per_second/number_of_bowties)*print_species.ionenergies[1]*constants.electron_volt)

total_ionization_power = per_chamber_ionization_power*number_of_bowties

ionization_chamber_area = math.pi*(ionization_chamber_radius**2.0)

print("Ionization power: per ~%0.5g W, total %0.5g W" % (per_chamber_ionization_power,total_ionization_power))

Ionization power: per ~0.0029729 W, total 59.458 W


ICP acceleration field, (line integral of Maxwell's second equation, eq. 4.5-3 of "Fundamentals"):

In [111]:
# (-(1.0j*2.0*math.pi*ICP_ionization_frequency)/2.0)*

 ## Assembly 
 
 Charge assembly energy, circular approximation (eq. 8.7 FLP Vol II):
 
 TODO: add cylindrical eq. - this is horribly inaccurate, but at least it's an upper bound.
 
 FIXME:
 
 Total charge:
 $$ Q_T = \text{pL}\pi\text{R}^2$$
 
 
 
 

In [112]:
ionization_chamber_volume = (math.pi*(ionization_chamber_radius**2))*bowtie_rms_neutral_velocity


per_chamber_assembled_energy = (3.0/5.0)*(((atoms_per_second/number_of_bowties)* \
                                 (ionization_chamber_radius/bowtie_rms_neutral_velocity)*constants.e)**2.0) \
                                /(4.0*math.pi*constants.epsilon_0*ionization_chamber_radius)*(bowtie_rms_neutral_velocity/ionization_chamber_radius)

print("Charge assembly power: per ~%0.5g W, total %0.5g W" % (per_chamber_assembled_energy, per_chamber_assembled_energy*number_of_bowties))

Charge assembly power: per ~1.1693 W, total 23385 W


# Child-Langmuir Ion Current Limit

$$ \text{I} = \frac{4}{9}\epsilon_0 \left(\frac{-2q}{m} \right) ^{0.5} \frac{\text{V}^{1.5}}{ \text{D}^2} \text{A} $$

```
I = beam current, amps
V = acceleration voltage, volts
q = ion charge, C
m = ion mass, kg
D = gap between acceleration plates, m
A = area

```

Suggested reading:

Fundamentals of Electric Propulsion, "Basic plasma physics". ../references/Fundamentals.pdf

The Child-Langmuir Law, Stanford EDU, Lucas et al [1]

[2]

[1]: https://web.stanford.edu/~ajlucas/The%20Child-Langmuir%20Law.pdf

[2]: http://www.physics.csbsju.edu/370/thermionic.pdf

In [113]:
def child_langmuir_current(voltage, charge, mass, gap, area):
    return (4.0/9.0)*constants.epsilon_0*(((2.0*charge/(mass))**0.5)*((voltage**1.5)/(gap**2.0))) * area
                                   
per_chamber_CL_ion_current = child_langmuir_current(acceleration_voltage,constants.e,(print_species.mass*amu),acceleration_gap,ionization_chamber_area)
                                   
print("Child-langmuir ion current: %0.5g A/bowtie" % per_chamber_CL_ion_current)
print("Total CL current: %0.5g A" % (per_chamber_CL_ion_current*number_of_bowties))


Child-langmuir ion current: 0.0034066 A/bowtie
Total CL current: 68.133 A


# Acceleration & space charge


In [114]:
acceleration_velocity = math.sqrt((2.0*acceleration_voltage*constants.electron_volt)/(print_species.mass*amu))

print("Accel. vel: %0.5g m/s" % acceleration_velocity)
print("Accel. power per chamber: %0.5g W, total %0.5g W " % (acceleration_voltage*per_chamber_beam_current
                                                        ,acceleration_voltage*total_beam_current))

Accel. vel: 23920 m/s
Accel. power per chamber: 0.039733 W, total 794.66 W 


In [115]:
# Vkick

https://cds.cern.ch/record/1005034/files/p27.pdf 



https://arxiv.org/pdf/1401.3951.pdf



In [116]:
beam_exit_velocity = bowtie_rms_neutral_velocity+acceleration_velocity

focus_field = util.scharge_efield(per_chamber_beam_current,beam_exit_velocity,ionization_chamber_radius)
print("Required focus field: %0.5g V/m" % focus_field)

#sine focus voltage equation would be nice
print("Required focus voltage: ~%0.5g V" % (focus_field*(acceleration_gap)))


Required focus field: 1.1882e+05 V/m
Required focus voltage: ~29.704 V


# Warp acceleration gap sim

This required a modification to warp - line 28 of warp.py needs to have 
```
try:
    __IPYTHON__
    pass
except NameError:
    warpoptions.parse_args()
```

In [117]:
from warp import *

w3d.solvergeom = w3d.RZgeom

w3d.bound0 = dirichlet
w3d.boundnz = neumann
w3d.boundxy = neumann
# ---   for particles
top.pbound0 = absorb
top.pboundnz = absorb
top.prwall = ionization_chamber_radius

# --- Set field grid size
w3d.xmmin = -ionization_chamber_radius
w3d.xmmax = +ionization_chamber_radius
w3d.ymmin = -ionization_chamber_radius
w3d.ymmax = +ionization_chamber_radius
w3d.zmmin = 0.
w3d.zmmax = ionization_chamber_length

# --- Field grid dimensions - note that nx and ny must be even.
w3d.nx = w3d.ny = 32
w3d.nz = 32

f3d.mgtol = 1.e-1 # Multigrid solver convergence tolerance, in volts


plate = ZRoundedCylinderOut(radius=0.0001, length=0.0001, radius2=0.00001, voltage=100.0, zcent=0.01)

installconductor(plate,dfill=largepos)

solver = MultiGrid2D()
registersolver(solver)

package("w3d")
generate()

step()

getappliedfields(x=np.arange(-ionization_chamber_radius,ionization_chamber_radius,0.001),
                 y=np.arange(-ionization_chamber_radius,ionization_chamber_radius,0.001),
                 z=0.0)


 ***  particle simulation package W3D generating
 ---  Resetting lattice array sizes
 ---  Allocating space for particles
 ---  Loading particles
 ---  Setting charge density
 ---  done
 ---  Allocating Win_Moments
 ---  Allocating Z_Moments
 ---  Allocating Lab_Moments
 Atomic number of ion =  0.0000E+00
 Charge state of ion  =  0.0000E+00
 Initial X, Y emittances =  0.0000E+00,   0.0000E+00 m-rad
 Initial X,Y envelope radii  =  0.0000E+00,   0.0000E+00 m
 Initial X,Y envelope angles =  0.0000E+00,   0.0000E+00 rad
 Input beam current =  0.0000E+00 amps
 Current density =  0.0000E+00 amps/m**2
 Charge density =  0.0000E+00 Coul/m**3
 Number density =  0.0000E+00
 Plasma frequency     =  0.0000E+00 1/s
    times dt          =  0.0000E+00
    times quad period =  0.0000E+00
 Plasma period        =  6.2832E+36 s
 X-, Y-Thermal Velocities     =  0.0000E+00,   0.0000E+00 m/s
    times dt                  =  0.0000E+00,   0.0000E+00 m
    times dt/dx, dt/dy (X, Y) =  0.0000E+00,   0.0000E+0

(array([0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0.]))

# Recombination

## Child-Langmuir recombination current


In [118]:
per_chamber_CL_electron_current = child_langmuir_current(acceleration_voltage,constants.e,constants.electron_mass,acceleration_gap,ionization_chamber_area)

print("Child-langmuir electron current: %0.5g A/bowtie" % per_chamber_CL_electron_current )
print("Total CL e current: %0.5g A" % (per_chamber_CL_electron_current*number_of_bowties))


Child-langmuir electron current: 0.75551 A/bowtie
Total CL e current: 15110 A


## Hot cathode

# Part heating

The beam heat flux will cause a temperature gradient from the chilled build platform to the extreme end of the printed part. Assuming a constant cross section, the peak temperature will be

$$ \frac{\text{(P/A)}}{k}L = \Delta \text{T} \\ \Delta \text{T} + \text{T}_i = T $$

```
P = Power applied at surface, W
A = Area of the part, m^2
k = Thermal conductivity of material, W/m-K
L = Length, M
T = Change in temperature across material, K
Ti = Build platform temperature
```

Ionization power is likely emitted in the form of photons at recombination, and so should be neglected.


http://web.mit.edu/16.unified/www/SPRING/propulsion/notes/node116.html

TODO: 
- Add radiation here. A bit tricky, since the temperature's not constant.
- Add some kind of duty cycle approximation?
- cool graph

In [119]:
print_area = 0.01*0.1 #m^2
printed_object_height = 0.03 #m
build_platform_temperature = 200.0 #Kelvin - likely liquid-cooled 

printed_emissivity = 0.05
beam_power = (acceleration_voltage*total_beam_current)

part_peak_temperature = (((beam_power/print_area)/print_species.thermal_conductivity)*printed_object_height)+build_platform_temperature
print("Part peak temperature: %0.5g K" % (part_peak_temperature))


# per_bowtie_power = (bowtie_hot_area * printed_emissivity * constants.Stefan_Boltzmann * (bowtie_temperature**4.0))


Part peak temperature: 300.59 K


# A final tally

In [120]:
total_power_consumption = total_bowtie_power + (acceleration_voltage*total_beam_current) + total_ionization_power
print("Total power consumption: %0.5g W" % (total_power_consumption))

print("Print head area: %0.5g m^2 (a square %0.5g m to the side)" % ((ionization_chamber_area*number_of_bowties),((ionization_chamber_area*number_of_bowties)**0.5)))


Total power consumption: 4992.9 W
Print head area: 0.56549 m^2 (a square 0.75199 m to the side)


In [121]:
#economics!

cost_per_watt = 0.2/1000.0 #20 cents per kilowatt

consumable_cost = 200.0/1000.0 #dollars per 1000 hours

material_cost = 20.0/1000.0 #dollars per gram

watts_per_gram = total_power_consumption/print_speed_g_h

print("Print cost: %0.5g W/g, $%0.5g/100g" % ((watts_per_gram),(cost_per_watt*watts_per_gram*100.0+material_cost+consumable_cost)))


Print cost: 499.29 W/g, $10.206/100g
